In [1]:
import random
import math
from math import inf as infinity
import copy
import matplotlib.pyplot as plt
import torch
from torch import nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device
# torch.set_default_device(device)

'cpu'

In [9]:
Q_LEARNING = 1
OPPONENT = 2
import numpy as np
initial_epsilon = 1
# q_values = {}
win_rate_overtime = []
q_learning_score = 0
opponent_score = 0


size = int(input('Size of the board '))
n = int(input('How many components in a line to win '))
number_of_games = int(input('Number of matches '))

board = [[0 for _ in range(size)] for _ in range(size)]

def game_over(state):
    return wins(state, Q_LEARNING) or wins(state, OPPONENT)


def empty_cells(state):
    empty = []
    for x, row in enumerate(state):
        for y, col in enumerate(row):
            if col == 0:
                empty.append([x,y])
    return empty

def valid_move(x,y):
    global board
    if board[x][y] == 0:
        return True
    return False

def set_move(x,y,player):
    if valid_move(x,y):
        board[x][y] = player
        return True
    return False

def wins(state, player):
  # Check row
  for row in state:
    for i in range(len(row) - n + 1):
      if all(row[i + k] == player for k in range(n)):
        return True

  # Check col
  for i in range(len(state)):
    for j in range(len(state) - n + 1):
      if all(state[j + k][i] == player for k in range(n)):
        return True

  # Check diagonal (top-left to bottom-right)
  for i in range(len(state) - n + 1):
    for j in range(len(state[0]) - n + 1):
      if all(state[i + k][j + k] == player for k in range(n)):
        return True

  # Check diagonal (top-right to bottom-left)
  for i in range(len(state) - n + 1):
    for j in range(n - 1, len(state[0])):
      if all(state[i + k][j - k] == player for k in range(n)):
        return True

  return False


def find_open_n(state, player):
    # Check row
    for row in state:
        for i in range(len(row) - n + 1):
            if all(row[i + k] == player for k in range(n)):
                if (i - 1 >= 0 and row[i - 1] == 0) and (i + n < len(row) and row[i + n] == 0):
                    return True

    # Check col
    for i in range(len(state)):
        for j in range(len(state) - n + 1):
            if all(state[j + k][i] == player for k in range(n)):
                if (j - 1 >= 0 and state[j - 1][i] == 0) and (j + n < len(state) and state[j + n][i] == 0):
                    return True

    # Check diagonal (top-left to bottom-right)
    for i in range(len(state) - n + 1):
        for j in range(len(state[0]) - n + 1):
            if all(state[i + k][j + k] == player for k in range(n)):
                if (i - 1 >= 0 and j - 1 >= 0 and state[i - 1][j - 1] == 0) and (i + n < len(state) and j + n < len(state[0]) and state[i + n][j + n] == 0):
                    return True

    # Check diagonal (top-right to bottom-left)
    for i in range(len(state) - n + 1):
        for j in range(n - 1, len(state[0])):
            if all(state[i + k][j - k] == player for k in range(n)):
                if (i - 1 >= 0 and j + 1 < len(state[0]) and state[i - 1][j + 1] == 0) and (i + n < len(state) and j - n >= 0 and state[i + n][j - n] == 0):
                    return True

    return False

def check_3(state, player):
    score = 0
    # Check row
    for row in state:
        for i in range(len(row) - 2):
            if all(row[i + k] == player for k in range(3)):
                score += 45

    # Check col
    for i in range(len(state)):
        for j in range(len(state) - 2):
            if all(state[j + k][i] == player for k in range(3)):
                score += 45

    # Check diagonal (top-left to bottom-right)
    for i in range(len(state) - 2):
        for j in range(len(state[0]) - 2):
            if all(state[i + k][j + k] == player for k in range(3)):
                score += 45

    # Check diagonal (top-right to bottom-left)
    for i in range(len(state) - 2):
        for j in range(2, len(state[0])):
            if all(state[i + k][j - k] == player for k in range(3)):
                score += 45

    return score

def check_2(state, player):
    score = 0
    # Check row
    for row in state:
        for i in range(len(row) - 1):
            if all(row[i + k] == player for k in range(2)):
                score += 20

    # Check col
    for i in range(len(state)):
        for j in range(len(state) - 1):
            if all(state[j + k][i] == player for k in range(2)):
                score += 20

    # Check diagonal (top-left to bottom-right)
    for i in range(len(state) - 1):
        for j in range(len(state[0]) - 1):
            if all(state[i + k][j + k] == player for k in range(2)):
                score += 20

    # Check diagonal (top-right to bottom-left)
    for i in range(len(state) - 1):
        for j in range(1, len(state[0])):
            if all(state[i + k][j - k] == player for k in range(2)):
                score += 20

    return score

def def_3(state, player, opponent):
    score = 0
    # Check row
    for row in state:
        for i in range(len(row) - 2):
            if all(row[i + k] == player for k in range(3)):
                # Check bounds for left and right blocking
                left_blocked = i > 0 and row[i - 1] == opponent
                right_blocked = i + 3 < len(row) and row[i + 3] == opponent
                if left_blocked or right_blocked:
                    score += 60

    # Check col
    for i in range(len(state)):
        for j in range(len(state) - 2):
            if all(state[j + k][i] == player for k in range(3)):
                # Check bounds for top and bottom blocking
                top_blocked = j > 0 and state[j - 1][i] == opponent
                bottom_blocked = j + 3 < len(state) and state[j + 3][i] == opponent
                if top_blocked or bottom_blocked:
                    score += 60

    # Check diagonal (top-left to bottom-right)
    for i in range(len(state) - 2):
        for j in range(len(state[0]) - 2):
            if all(state[i + k][j + k] == player for k in range(3)):
                # Check bounds for top-left and bottom-right blocking
                top_left_blocked = i > 0 and j > 0 and state[i - 1][j - 1] == opponent
                bottom_right_blocked = i + 3 < len(state) and j + 3 < len(state[0]) and state[i + 3][j + 3] == opponent
                if top_left_blocked or bottom_right_blocked:
                    score += 60

    # Check diagonal (top-right to bottom-left)
    for i in range(len(state) - 2):
        for j in range(2, len(state[0])):
            if all(state[i + k][j - k] == player for k in range(3)):
                # Check bounds for top-right and bottom-left blocking
                top_right_blocked = i > 0 and j + 1 < len(state[0]) and state[i - 1][j + 1] == opponent
                bottom_left_blocked = i + 3 < len(state) and j - 3 >= 0 and state[i + 3][j - 3] == opponent
                if top_right_blocked or bottom_left_blocked:
                    score += 60

    return score

def def_2(state, player, opponent):
    score = 0
    # Check row
    for row in state:
        for i in range(len(row) - 1):
            if all(row[i + k] == player for k in range(2)):
                # Check bounds for left and right blocking
                left_blocked = i > 0 and row[i - 1] == opponent
                right_blocked = i + 2 < len(row) and row[i + 2] == opponent
                if left_blocked or right_blocked:
                    score += 40

    # Check col
    for i in range(len(state)):
        for j in range(len(state) - 1):
            if all(state[j + k][i] == player for k in range(2)):
                # Check bounds for top and bottom blocking
                top_blocked = j > 0 and state[j - 1][i] == opponent
                bottom_blocked = j + 2 < len(state) and state[j + 2][i] == opponent
                if top_blocked or bottom_blocked:
                    score += 40

    # Check diagonal (top-left to bottom-right)
    for i in range(len(state) - 1):
        for j in range(len(state[0]) - 1):
            if all(state[i + k][j + k] == player for k in range(2)):
                # Check bounds for top-left and bottom-right blocking
                top_left_blocked = i > 0 and j > 0 and state[i - 1][j - 1] == opponent
                bottom_right_blocked = i + 2 < len(state) and j + 2 < len(state[0]) and state[i + 2][j + 2] == opponent
                if top_left_blocked or bottom_right_blocked:
                    score += 40

    # Check diagonal (top-right to bottom-left)
    for i in range(len(state) - 1):
        for j in range(1, len(state[0])):
            if all(state[i + k][j - k] == player for k in range(2)):
                # Check bounds for top-right and bottom-left blocking
                top_right_blocked = i > 0 and j + 1 < len(state[0]) and state[i - 1][j + 1] == opponent
                bottom_left_blocked = i + 2 < len(state) and j - 2 >= 0 and state[i + 2][j - 2] == opponent
                if top_right_blocked or bottom_left_blocked:
                    score += 40

    return score

def find_open_three(state, player):
    score = 0
    # Check row
    for row in state:
        for i in range(len(row) - 2):
            if all(row[i + k] == player for k in range(3)):
                if (i-1 >= 0 and row[i-1] == 0) and (i+3 < len(row) and row[i+3] == 0):
                    score += 100  # Cộng điểm cho "open three"

    # Check col
    for i in range(len(state)):
        for j in range(len(state) - 2):
            if all(state[j + k][i] == player for k in range(3)):
                if (j-1 >= 0 and state[j-1][i] == 0) and (j+3 < len(state) and state[j+3][i] == 0):
                    score += 100  # Cộng điểm cho "open three"

    # Check diagonal (top-left to bottom-right)
    for i in range(len(state) - 2):
        for j in range(len(state[0]) - 2):
            if all(state[i + k][j + k] == player for k in range(3)):
                if (i-1 >= 0 and j-1 >= 0 and state[i-1][j-1] == 0) and (i+3 < len(state) and j+3 < len(state[0]) and state[i+3][j+3] == 0):
                    score += 100  # Cộng điểm cho "open three"

    # Check diagonal (top-right to bottom-left)
    for i in range(len(state) - 2):
        for j in range(2, len(state[0])):
            if all(state[i + k][j - k] == player for k in range(3)):
                if (i-1 >= 0 and j+1 < len(state[0]) and state[i-1][j+1] == 0) and (i+3 < len(state) and j-3 >= 0 and state[i+3][j-3] == 0):
                    score += 100  # Cộng điểm cho "open three"

    return score

# should be exclusive for 5x5 board
def find_open_four(state, player):
    score = 0
    # Check row
    for row in state:
        for i in range(len(row) - 3):
            if all(row[i + k] == player for k in range(4)):
                if (i-1 >= 0 and row[i-1] == 0) and (i+4 < len(row) and row[i+4] == 0):
                    score += 500  # Cộng điểm cho "open four"

    # Check col
    for i in range(len(state)):
        for j in range(len(state) - 3):
            if all(state[j + k][i] == player for k in range(4)):
                if (j-1 >= 0 and state[j-1][i] == 0) and (j+4 < len(state) and state[j+4][i] == 0):
                    score += 500  # Cộng điểm cho "open four"

    # Check diagonal (top-left to bottom-right)
    for i in range(len(state) - 3):
        for j in range(len(state[0]) - 3):
            if all(state[i + k][j + k] == player for k in range(4)):
                if (i-1 >= 0 and j-1 >= 0 and state[i-1][j-1] == 0) and (i+4 < len(state) and j+4 < len(state[0]) and state[i+4][j+4] == 0):
                    score += 500  # Cộng điểm cho "open four"

    # Check diagonal (top-right to bottom-left)
    for i in range(len(state) - 3):
        for j in range(3, len(state[0])):
            if all(state[i + k][j - k] == player for k in range(4)):
                if (i-1 >= 0 and j+1 < len(state[0]) and state[i-1][j+1] == 0) and (i+4 < len(state) and j-4 >= 0 and state[i+4][j-4] == 0):
                    score += 500  # Cộng điểm cho "open four"

    return score

def find_dangerous_move(state, player):
    # Kiểm tra hàng ngang
    score = 0
    for row in state:
        for i in range(len(row) - 3):
            # Mẫu 1 1 0 1
            if (row[i] == player and row[i + 1] == player and row[i + 2] == 0 and row[i + 3] == player):
                if (i - 1 >= 0 and row[i - 1] == 0) and (i + 4 < len(row) and row[i + 4] == 0):
                    score += 300
            # Mẫu 1 0 1 1
            if (row[i] == player and row[i + 1] == 0 and row[i + 2] == player and row[i + 3] == player):
                if (i - 1 >= 0 and row[i - 1] == 0) and (i + 4 < len(row) and row[i + 4] == 0):
                    score += 300

    # Kiểm tra cột dọc
    for i in range(len(state)):
        for j in range(len(state) - 3):
            # Mẫu 1 1 0 1
            if (state[j][i] == player and state[j + 1][i] == player and state[j + 2][i] == 0 and state[j + 3][i] == player):
                if (j - 1 >= 0 and state[j - 1][i] == 0) and (j + 4 < len(state) and state[j + 4][i] == 0):
                    score += 300
            # Mẫu 1 0 1 1
            if (state[j][i] == player and state[j + 1][i] == 0 and state[j + 2][i] == player and state[j + 3][i] == player):
                if (j - 1 >= 0 and state[j - 1][i] == 0) and (j + 4 < len(state) and state[j + 4][i] == 0):
                    score += 300

    # Kiểm tra đường chéo từ trái trên xuống phải dưới
    for i in range(len(state) - 3):
        for j in range(len(state[0]) - 3):
            # Mẫu 1 1 0 1
            if (state[i][j] == player and state[i + 1][j + 1] == player and state[i + 2][j + 2] == 0 and state[i + 3][j + 3] == player):
                if (i - 1 >= 0 and j - 1 >= 0 and state[i - 1][j - 1] == 0) and (i + 4 < len(state) and j + 4 < len(state[0]) and state[i + 4][j + 4] == 0):
                    score += 300
            # Mẫu 1 0 1 1
            if (state[i][j] == player and state[i + 1][j + 1] == 0 and state[i + 2][j + 2] == player and state[i + 3][j + 3] == player):
                if (i - 1 >= 0 and j - 1 >= 0 and state[i - 1][j - 1] == 0) and (i + 4 < len(state) and j + 4 < len(state[0]) and state[i + 4][j + 4] == 0):
                    score += 300

    # Kiểm tra đường chéo từ phải trên xuống trái dưới
    for i in range(len(state) - 3):
        for j in range(3, len(state[0])):
            # Mẫu 1 1 0 1
            if (state[i][j] == player and state[i + 1][j - 1] == player and state[i + 2][j - 2] == 0 and state[i + 3][j - 3] == player):
                if (i - 1 >= 0 and j + 1 < len(state[0]) and state[i - 1][j + 1] == 0) and (i + 4 < len(state) and j - 4 >= 0 and state[i + 4][j - 4] == 0):
                    score += 300
            # Mẫu 1 0 1 1
            if (state[i][j] == player and state[i + 1][j - 1] == 0 and state[i + 2][j - 2] == player and state[i + 3][j - 3] == player):
                if (i - 1 >= 0 and j + 1 < len(state[0]) and state[i - 1][j + 1] == 0) and (i + 4 < len(state) and j - 4 >= 0 and state[i + 4][j - 4] == 0):
                    score += 300

    return score 



def random_move(state):
    actions = empty_cells(state)
    return random.choice(actions)

def evaluate(state):
    score = 0
    if wins(state,Q_LEARNING):
        score += 2000
    if wins(state, OPPONENT):
        score -= 1500
    if find_open_n(state, Q_LEARNING):
        score += 800
    if find_open_n(state,OPPONENT):
        score -= 600
    connect_2 = check_2(state, Q_LEARNING)
    score += connect_2
    connect_3 = check_3(state, Q_LEARNING)
    score += connect_3
    connect_2 = check_2(state, OPPONENT)
    score -= connect_2
    connect_3 = check_3(state, OPPONENT)
    score -= connect_3
    defend = def_3(state, OPPONENT, Q_LEARNING)
    score += defend
    defend = def_2(state, OPPONENT, Q_LEARNING)
    score += defend
    open_three = find_open_three(state, Q_LEARNING)
    score += open_three
    open_four = find_open_four(state, Q_LEARNING)
    score += open_four
    dangerous = find_dangerous_move(state, Q_LEARNING)
    score += dangerous
    open_three = find_open_three(state, OPPONENT)
    score -= open_three
    open_four = find_open_four(state, OPPONENT)
    score -= open_four
    dangerous = find_dangerous_move(state, OPPONENT)
    score -= dangerous
    
    return score

def render(state,c_choice, h_choice):
    choices = {
        1 : h_choice,
        2 : c_choice,
        0: ' '
    }
    line = '---------------------------------------------'
    print(line)
    for row in state:
        for col in row:
            symbol = choices[col]
            print('|',symbol,'|',end='')
        print()
        print(line)

def copy_board(state):
    return copy.deepcopy(state)

def check_winner(state):
    global q_learning_score, opponent_score
    if wins(state, Q_LEARNING):
        # print("Q LEARNING WINS")
        q_learning_score += 1
    elif wins(state, OPPONENT):
        # print("RANDOM OPPONENT WINS")
        opponent_score += 1
    # else:
    #     # print("IT'S A DRAW")
    return

In [10]:
class DQN(nn.Module):
    def __init__(self, state_dims):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features=state_dims, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=size*size)
        )
    def forward(self, x):
        return self.block(x)

q_network = DQN(size * size)
print(f'q_network is running on {next(q_network.parameters()).device}')
target_q_network = copy.deepcopy(q_network).eval()

q_network is running on cpu


In [11]:
from torch.nn import functional as F
optimizer = torch.optim.Adam(q_network.parameters(),
                             lr = 0.001)

In [12]:
class ReplayMemory:
    def __init__(self, capacity=1000000):
        self.capacity = capacity
        self.memory = []
        self.position = 0
    
    def insert(self, transition):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = transition
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        assert self.can_sample(batch_size)
        batch = random.sample(self.memory, batch_size)
        batch = zip(*batch)
        return [torch.cat(items) for items in batch]
    
    def can_sample(self, batch_size):
        return len(self.memory) >= batch_size * 10
    
    def __len__(self):
        return len(self.memory)

memory = ReplayMemory()

In [13]:
from tqdm.auto import tqdm
from timeit import default_timer as timer

In [14]:
def reset():
    global board
    board = [[0 for _ in range(size)] for _ in range(size)]
    
def encode_state(state):
    list_state = [item for sublist in state for item in sublist]
    torch_state = torch.tensor(list_state, dtype= torch.float32)
    return torch_state

def exploratory_policy(state, epsilon):
    actions = empty_cells(state)
    if random.random() < epsilon:
        # print('explore move')
        return random.choice(actions)
    else:
        # print('optimal move')
        encoded_state = encode_state(state)
        actions_logits = q_network(encoded_state.unsqueeze(dim = 0)).detach()
        for idx in range(actions_logits.size(1)):
            row = idx // size
            col = idx % size
            if not valid_move(row, col):
                actions_logits[0, idx] = -infinity 

        while True:
            best_action = torch.argmax(actions_logits, dim=1).item()
            row = best_action // size
            col = best_action % size
            
            if valid_move(row, col):
                # print(f'X move: {best_action}')
                return [row, col]
            else:
                actions_logits[0, best_action] = -infinity

def check_end():
    return game_over(board) or len(empty_cells(board)) == 0

def game_with_q_learning(func, c_choice, h_choice, episode, decay_rate=0.01, alpha=0.1, gamma=0.5, batch_size = 32):
    epsilon = initial_epsilon * math.e ** (-decay_rate * episode)
    # print(f'current epsilon: {epsilon}')
    while not check_end():
        state = encode_state(board).unsqueeze(0)
        action = exploratory_policy(board, epsilon)
        x, y = action[0], action[1]
        action = x*size + y
        action = torch.tensor(action, dtype= torch.long).unsqueeze(0).unsqueeze(0)
        set_move(x, y, Q_LEARNING)
        reward = evaluate(board)
        reward = torch.tensor(reward, dtype= torch.float32).unsqueeze(0).unsqueeze(0)
        # render(board, c_choice, h_choice)
        
        if check_end():
            done = True
            done = torch.tensor(done, dtype=torch.bool).unsqueeze(0).unsqueeze(0)
            memory.insert([state, action, reward, done, state])
            break
        else:
            move = func(board)
            set_move(move[0], move[1], OPPONENT)
            # render(board, c_choice, h_choice)
            next_state = encode_state(board).unsqueeze(0)
            done = check_end()
            done = torch.tensor(done, dtype=torch.bool).unsqueeze(0).unsqueeze(0)
            memory.insert([state, action, reward, done, next_state])
        
        if memory.can_sample(batch_size):
            torch.manual_seed(42)
            torch.cuda.manual_seed(42)
            # print('condition for deep learning accquired')
            state_b, action_b, reward_b, done_b, next_state_b = memory.sample(batch_size)
            qsa_b = q_network(state_b).gather(1, action_b.long())
            
            next_qsa_b = target_q_network(next_state_b)
            next_qsa_b = torch.max(next_qsa_b, dim=-1, keepdim=True)[0]
            target_b = reward_b + ~done_b * gamma * next_qsa_b
            
            loss = F.mse_loss(qsa_b, target_b)
            q_network.zero_grad()
            loss.backward()
            optimizer.step()
            
        
    if episode % 10 == 0:
        target_q_network.load_state_dict(q_network.state_dict())
    check_winner(board)
    reset()

def main():
    global board
    random.seed(42)
    h_choice = 'X'
    c_choice = 'O'
    q_network.train()
    reset()
    start_time = timer()
    for i in tqdm(range(number_of_games), disable = False):
        game_with_q_learning(random_move, c_choice, h_choice, i)
        win_rate = q_learning_score / (i + 1)
        win_rate_overtime.append(win_rate)
        # print(f'{i}th game')
    end_time = timer()
    print(f'training time {start_time - end_time}')
    print(f'final score: dynamic: {q_learning_score} - opponent: {opponent_score}')
    print(f'first 5 index of win rate over time: {win_rate_overtime[0:5]}')
    print(f'last 5 index of win rate over time: {win_rate_overtime[-5:]}')
if __name__ == '__main__':
    main()

  0%|          | 0/1000 [00:00<?, ?it/s]

training time -3.663210792001337
final score: dynamic: 788 - opponent: 128
first 5 index of win rate over time: [1.0, 0.5, 0.3333333333333333, 0.5, 0.4]
last 5 index of win rate over time: [0.7871485943775101, 0.7873620862587764, 0.7875751503006012, 0.7877877877877878, 0.788]
